#### Importação das libs

In [1]:
import streamlit as st
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd
import pandas_gbq as pdb
import tqdm
import numpy as np


#### Fazer o download das bases - dicionario e microdados

In [ ]:
# Buscar credenciais do Big Query
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
client = bigquery.Client(credentials=credentials)

### Download das bases
#Dicionario
df_dicionario = pdb.read_gbq("select * from basedosdados.br_ibge_pnad_covid.dicionario", project_id="postech-414800", credentials=credentials)
df_dicionario.to_csv('.\\bases\\dicionario.csv',sep=";", header=True)

#microdados
df_ano_mes = pdb.read_gbq("select ano, mes, count(*) as cnt from basedosdados.br_ibge_pnad_covid.microdados group by ano, mes", project_id="postech-414800", credentials=credentials)
df_ano_mes.sort_values(by=['mes'], inplace=True)

lst_df = []
for k,i in df_ano_mes.iterrows():
    print(f"{i['ano']} - {i['mes']}")
    df_tmp = pdb.read_gbq(f"select * from basedosdados.br_ibge_pnad_covid.microdados where ano = {i['ano']} and mes = {i['mes']}", project_id="postech-414800", credentials=credentials)
    lst_df.append(df_tmp)

df_concat = pd.concat(lst_df)
df_concat.to_csv('.\\bases\\microdados.csv',sep=";", header=True)

#### Atribuir o valor corresponde no dicionário aos campos da base de microdados

In [ ]:
#ler bases e processar o dicionario
df_dic = pd.read_csv(r'.\bases\dicionario.csv', sep=";")
df_m = pd.read_csv(r'.\bases\microdados.csv', sep=";")

df_m.drop(columns=['Unnamed: 0'], inplace=True)

#fazer loop pelos campos
#atribuir o 'valor' ao nome do campo - ex df['capital'] = df['valor']
df_m_trat = df_m
for c in df_dic['nome_coluna'].unique():
    print(f"Campo processado: {c}")
    df_d = df_dic.query(f"nome_coluna == '{c}'")[['chave','valor']]
    df_m_trat = df_m_trat.merge(df_d, how='left', left_on=c, right_on='chave')
    df_m_trat[c] = df_m_trat['valor']
    df_m_trat.drop(columns=['chave','valor'], inplace=True)


#### Enriquecimento da base

- Com base na UF, incluir nome do estado
- Com base no nome do estado, incluir a região 
- Com base na idade, aplicar uma faixa etária
- Classificar a escolaridade (campo a005)
- Agrupar as questões sobre os sintomas (b0011 até b00113) em um único campo

In [23]:
estados = {
    "AC": "Acre",
    "AL": "Alagoas",
    "AP": "Amapá",
    "AM": "Amazonas",
    "BA": "Bahia",
    "CE": "Ceará",
    "DF": "Distrito Federal",
    "ES": "Espírito Santo",
    "GO": "Goiás",
    "MA": "Maranhão",
    "MT": "Mato Grosso",
    "MS": "Mato Grosso do Sul",
    "MG": "Minas Gerais",
    "PA": "Pará",
    "PB": "Paraíba",
    "PR": "Paraná",
    "PE": "Pernambuco",
    "PI": "Piauí",
    "RJ": "Rio de Janeiro",
    "RN": "Rio Grande do Norte",
    "RS": "Rio Grande do Sul",
    "RO": "Rondônia",
    "RR": "Roraima",
    "SC": "Santa Catarina",
    "SP": "São Paulo",
    "SE": "Sergipe",
    "TO": "Tocantins"
}

regioes = {
    "Acre": "Norte",
    "Alagoas": "Nordeste",
    "Amapá": "Norte",
    "Amazonas": "Norte",
    "Bahia": "Nordeste",
    "Ceará": "Nordeste",
    "Distrito Federal": "Centro-Oeste",
    "Espírito Santo": "Sudeste",
    "Goiás": "Centro-Oeste",
    "Maranhão": "Nordeste",
    "Mato Grosso": "Centro-Oeste",
    "Mato Grosso do Sul": "Centro-Oeste",
    "Minas Gerais": "Sudeste",
    "Pará": "Norte",
    "Paraíba": "Nordeste",
    "Paraná": "Sul",
    "Pernambuco": "Nordeste",
    "Piauí": "Nordeste",
    "Rio de Janeiro": "Sudeste",
    "Rio Grande do Norte": "Nordeste",
    "Rio Grande do Sul": "Sul",
    "Rondônia": "Norte",
    "Roraima": "Norte",
    "Santa Catarina": "Sul",
    "São Paulo": "Sudeste",
    "Sergipe": "Nordeste",
    "Tocantins": "Norte"
}

# Crie um dicionário com os limites de idade para cada faixa etária
faixas_etarias = {
    "0 a 12 anos": (0, 12),
    "13 a 18 anos": (13, 18),
    "19 a 24 anos": (19, 24),
    "25 a 34 anos": (25, 34),
    "35 a 44 anos": (35, 44),
    "45 a 54 anos": (45, 54),
    "55 a 64 anos": (55, 64),
    "65 anos ou mais": (65, 200)
}

# Crie uma nova coluna com a sugestão de faixa etária
def faixa_etaria(idade):
    for faixa, limites in faixas_etarias.items():
        if limites[0] <= idade <= limites[1]:
            return faixa
    return None

escolaridade = {'Fundamental incompleto':2, 'Médio completo':5, 'Médio incompleto':4,
       'Pós-graduação, mestrado ou doutorado':8, 'Sem instrução':1,
       'Fundamental completa':3, 'Superior incompleto':6, 'Superior completo':7}

df_m_trat["faixa_etaria"] = df_m_trat["a002"].apply(faixa_etaria)
df_m_trat["estado"] = df_m_trat["sigla_uf"].map(estados)
df_m_trat["regiao"] = df_m_trat["estado"].map(regioes)
df_m_trat["escolaridade"] = df_m_trat["a005"].map(escolaridade)
df_m_trat['sintomas'] = np.any([df_m_trat['b0011'] == 'Sim',df_m_trat['b0012'] == 'Sim',df_m_trat['b0013'] == 'Sim',df_m_trat['b0014'] == 'Sim',df_m_trat['b0015'] == 'Sim',
                               df_m_trat['b0016'] == 'Sim',df_m_trat['b0017'] == 'Sim',df_m_trat['b0018'] == 'Sim',df_m_trat['b0019'] == 'Sim',df_m_trat['b00110'] == 'Sim',
                               df_m_trat['b00111'] == 'Sim',df_m_trat['b00112'] == 'Sim',df_m_trat['b00113'] == 'Sim'], axis=0)

#### Exportar como parquet

- Facilita a importação no Big Query


In [19]:
df_m_trat.to_parquet(r'.\bases\microdados.parquet', index=False)